# Plotting FVCOM4 NetCDF unstructured grid output using datashader and DynamicMap
**Author: Jun Sasaki; coded on June 30, 2018, updated on October 9, 2022** <br>
- datashaderでカラーマップを描き，meshはholoviewsのtrimesh.edgepathsで描く
- 時間ステップを含む平面2次元と空間3次元データを切り出し，カラーマップを描けるようにした
- 項目，時刻，σ層等を与えて，一気に描画するClass methodを作成した
- geoviews対応が課題
### The following warnings may appear.
- WARNING:Fiona:GDAL data files not located, GDAL_DATA not set
- WARNING:Fiona:PROJ data files not located, PROJ_LIB not set

## Install guide
Recommended to install on Linux. This notebook is tested on Ubuntu 20.04LTS on WSL2.
- Create `./png/` directory.
### Installing Firefox
Firefox or chronium is required for interactive plotting. Firefox may be easier to use on Linux. 
```bash
$ sudo apt update
$ sudo apt install firefox
```
### Installing packages on Miniconda Python confirmed on Oct 8, 2022
There are missing packages in the following list, which should be updated.
```bash
$ conda config --add channels conda-forge
$ conda install pandas geopandas matplotlib netcdf4 xarray
$ conda install cartopy
$ conda install hvplot
$ conda install pygmt
$ conda install jupyterlab
$ conda install geoviews
$ conda install selenium
$ conda install firefox geckodriver
```
### Exporting from html to png using pyppeteer
```bash
$ conda install pyppeteer
```

In [ ]:
import sys
import os
from functools import partial, partialmethod
import numpy as np
import pandas as pd
import pyproj
### import xarray as xr
import netCDF4
### import matplotlib.pyplot as plt
import matplotlib.tri as tri
import holoviews as hv
from holoviews import opts
#import datashader as ds
import datashader.utils as du
#import datashader.transfer_functions as tf
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize,\
    spread, aggregate, regrid
from holoviews.operation import decimate
import holoviews.plotting.mpl
#from holoext.xbokeh import Mod
import geoviews as gv
from bokeh import models
import subprocess
from bokeh.io import export_png
#import pdfkit
#from holoviews.plotting.bokeh.element import (line_properties, fill_properties, text_properties)
hv.extension('bokeh', 'matplotlib')

# Converting between orthogonal and geographic coordinates
- 平面直角座標と地理座標の変換を行う関数定義

In [ ]:
def proj_trans(df=None, col_x='Longitude', col_y='Latitude', epsg0="2451", epsg1="4612"):
    '''
    Transform coordinates of pandas.DataFrame (first two columns of (x, y)) 
    from epsg0 to epsg1; then the column names of (x, y) are renamed to (col_x, col_y),
    and return the DataFrame
    
    Parameters
    -------------
    df : pandas.DataFrame
    col_x : str
        x coordinate name
    col_y : str
        y coordinate name
    epsg0 : str
        original epsg code
    epsg1 : str
        transformed epsg code

    returns
    -------
    transformed pandas.DataFrame
    '''

    EPSG0 = pyproj.Proj("+init=EPSG:" + epsg0)
    EPSG1 = pyproj.Proj("+init=EPSG:" + epsg1)
    x1,y1 = pyproj.transform(EPSG0, EPSG1, df.iloc[:,0].values, df.iloc[:,1].values)
    ### print(len(x1))
    ### print(len(df))
    df[df.columns[0]]=x1
    df[df.columns[1]]=y1
    df.rename(columns = {df.columns[0]:col_x, df.columns[1]:col_y}, inplace=True)
    return df

## xlimとylimを設定するとInteractiveに解像度が変化しなくなる
- xlimとylimを固定して与える場合と与えない場合を分けて作っておく必要がある．前者は静的に扱い，後者は動的に扱うことを前提とする
- 一つの時間ステップtimeを引数として，基本的な描画を担う，関数time_zeta(time)を定義します．この段階でDatashaderを適用してはいけません．Datashaderはdynamic mapの一つで，後でDynamicMapを適用すると，DynamicMapのネストはサポートされていないというエラーとなるためです．さらに，この段階では.redim.range()を適用せず，描画段階で適用することで，フレキシビリティを確保します．
- 2d mapにおいて，cmapはrasterizeの段階で適用する必要がありそう．opts.Imageやopts.TriMeshでは適用できなかった．datashaderのオプションの与え方について要検討
## Application of `p.opts()`
- Application of `p.opts()` after creating p may be a good strategy to simplify the code. The Class FVCOM2D only creates simple plot instance `p`, followed by `p.opts()` for decorating the plot.

In [ ]:
class Fvcom2D:
    '''
    Postprocessing for FVCOM 4 netcdf output in its original coordinates
    '''
    def __init__(self, ncfile_path='tst_0001.nc', 
                 obcfile_path='tst_obc.dat', m_to_km=True, offset=False):
        '''
        Parameters
        ----------
        ncfile_path : str
            FVCOM output netcdf file
        obcfile_path : str
            FVCOM casename_obc.dat file
        m_to_km : bool
            = True if converting x and y axis units from m to km.
        offset : bool
           offset = True if the left-bottom corner is set to the origin.

        Returns
        -------
        Instance of class FVCOM2D
        '''

        self.fvcom_nc=ncfile_path
        if os.path.isfile(self.fvcom_nc):
            self.FVCOM = netCDF4.Dataset(self.fvcom_nc, 'r')
        else:
            print(f"ERROR: File {self.fvcom_nc} does not exit.")
            sys.exit()
        self.variables = self.FVCOM.variables
        print(f"Dictionaly keys of netCDF4.Dataset = {self.variables.keys()}")
        self.fobc=obcfile_path # = None if not exist
        if os.path.isfile(self.fobc):
            df = pd.read_csv(self.fobc, header=None, skiprows=1, delim_whitespace=True)
            ### -1 because index in FVCOM starts from 1 while from 0 in Python
            self.node_bc = df.iloc[:,1].values - 1
            print(f"Open boundary nodes = {self.node_bc}")
        else:
            print(f"ERROR: File {self.fobc} does not exit.")
            sys.exit()            
        self.x, self.y = self.variables['x'][:], self.variables['y'][:] ### at node
        self.xc, self.yc = self.variables['xc'][:], self.variables['yc'][:]   ### at cell center
        # Convert axis units from m to km
        if m_to_km:
            self.x /= 1000.0; self.y /= 1000.0; self.xc /= 1000.0; self.yc /= 1000.0
        # Set the left-bottom corner as the coordinate origin. 
        if offset:
            xmin = self.x.min(); ymin = self.y.min()
            self.x -= xmin; self.y -= ymin; self.xc -= xmin; self.yc -= ymin
        # Gets sigma coordinate values
        self.siglay, self.siglev = self.variables['siglay'][:], self.variables['siglev'][:]
        # Get time variables
        self.iint = self.variables['iint'][:]
        self.time = self.variables['time'][:]
        self.Itime = self.variables['Itime'][:]
        self.Itime2 = self.variables['Itime2'][:]
        # Gets bathymetry
        self.z = self.variables['h'][:]
        # Gets verts = [(x0,y0,h0),[x1,y1,h1],...,[xn,yn,hn]]
        verts = [(xi, yi, hi) for xi, yi, hi in zip(self.x.data, self.y.data, self.z.data)]
        self.verts = pd.DataFrame(verts, columns=['x', 'y', 'z'])
        # Gets connectivity array nv (3 node numbers of element i)
        nv = self.variables['nv'][:].T - 1
        self.triang = tri.Triangulation(self.x, self.y, triangles=nv)
        # Since node indexes in an element are defined clockwise in FVCOM,
        #     change them to counterclockwise, which is the matplotlib.tri specification.
        # FVCOMでは時計回りに定義されているので，matplotlib.triの仕様である反時計回りに変更する．
        # node番号自体は不変．
        self.nv=nv[:,::-1]
        self.tris = pd.DataFrame(self.nv, columns=['v0', 'v1', 'v2'])
        self.mesh = du.mesh(self.verts, self.tris) 
        self.trimesh = hv.TriMesh((self.tris, self.verts))
        # Element index of 3 neighbors of element i
        # 各セルには一般に3つのセルが隣接（境界セルはこの限りではない）
        self.nbe = np.array([[self.nv[n, j], self.nv[n, (j+2)%3]] \
            for n in range(len(self.triang.neighbors)) \
            for j in range(3) if self.triang.neighbors[n,j] == -1])

    @property
    def timestamp(self):
        '''
        Create time stamp list in ["D HH:MM:SS"]
        '''
        dayi = self.Itime
        hourf = self.Itime2 / 3600000
        houri = hourf.astype('int32')
        minf = (hourf - houri) * 60
        mini = minf.astype('int32')
        secf = (minf - mini) * 60
        seci = secf.astype('int32')
        return [f"{d} {h:02}:{m:02}:{s:02}" for d, h, m, s in zip(dayi, houri, mini, seci)]

    def plot_mesh(self, x_range=None, y_range=None, **kwargs):
        '''
        Plot mesh
        
        Parameters
        ----------
        x_range : tuple
            Plot range of x axis (xmin, xmax)
        y_range : tuple
            Plot range of y axis (ymin, ymax)
        '''

        # Gets **kwargs with default values
        #width=kwargs.get('width', 300)
        #height=kwargs.get('height', 300)
        line_color=kwargs.get('line_color', 'blue')
        line_width=kwargs.get('line_width', 0.1)

        # jsasaki debug
        #p = self.trimesh.edgepaths.options(width=width, height=height, 
        #                                   line_width=line_width, line_color=line_color)
        p = self.trimesh.edgepaths.options(line_width=line_width, line_color=line_color)

        # Set plot range
        if x_range is None and y_range is None:
            return p
        elif y_range is None:
            return p.redim.range(x=x_range)
        elif x_range is None:
            return p.redim.range(y=y_range)
        else:
            return p.redim.range(x=x_range, y=y_range)
    
    def plot_coastline(self, x_range=None, y_range=None, **kwargs):
        '''
        Plot coastline. May take much time for interactive plotting.

        Parameters
        ----------
        x_range : tuple
            Plot range in x-axis (xmin, xmax)
        y_range : tuple
            Plot range in y-aixs (ymin, ymax)
        '''

        # Gets **kwargs with default values
        #width=kwargs.get('width', 300)
        #height=kwargs.get('height', 300)
        color=kwargs.get('color', 'red')
        line_width=kwargs.get('line_width', 0.5)

        # paths = [[(x0s, y0s), (x0e, y0e)], ..., [(xns, yns), (xne, yne)]]
        paths = [[(self.x[self.nbe[m,:]][0], self.y[self.nbe[m,:]][0]), \
                  (self.x[self.nbe[m,:]][1], self.y[self.nbe[m,:]][1])] \
                 for m in range(len(self.nbe))]

        # jsasaki debug
        #p = hv.Path(paths).options(width=width, height=height, 
        #                           color=color, line_width=line_width)
        p = hv.Path(paths).options(color=color, line_width=line_width)

        if x_range is None and y_range is None:
            return p
        elif y_range is None:
            return p.redim.range(x=x_range)
        elif x_range is None:
            return p.redim.range(y=y_range)
        else:
            return p.redim.range(x=x_range, y=y_range)

    def get_val(self, item, time=None, sigma=None):
        '''
        Gets item values dependent considering its shape with specifying
        output time step (if time sereis data) and sigma level.

        Parameters
        ----------
        item : str
            Name of item in dictionary keys
        time : int
            Time series index
        sigma : int
            Layer number in sigma coordinates

        Returns
        -------
        pandas.DataFrame
        '''

        # Check whether item exists among the keys
        if not item in self.variables.keys():
            print(f"Error: Item {item} does not exit in keys.")
            sys.exit()
        val = self.FVCOM.variables[item]

        # Check the item dimensions
        if len(val.shape) == 1 : # 1-D item
            scalar = val
        elif len(val.shape) >= 2: # 2-D or 3-D item
            if time < 0 or time > val.shape[0]:
                print(f"ERROR: Time index = {time} is out of range.")
                print(f"Time index should be between 0 and {val.shape[0]-1}.")
                sys.exit()
            if len(val.shape) == 2: ### 2-D item
                scalar = val[time]
            elif len(val.shape) == 3: ### 3-D item
                if sigma < 0 or sigma > val.shape[1]:
                    print(f"ERROR: Sigma layer number = {sigma} is out of range.")
                    print(f"Sigma layer number should be between 0 and {val.shape[1]-1}.")
                    sys.exit()
                else:
                    scalar = val[time][sigma]
            else:
                print("ERROR: the shape is incorrect.")
                sys.exit()

        verts = [(xi, yi, hi) for xi, yi, hi in zip(self.x.data, self.y.data, scalar)]
        return pd.DataFrame(verts, columns=['x', 'y', item])

    def __plot_2dh(self, item, time, sigma):
        '''
        Internal method for converting pd.DataFrame of item to hv.TriMesh

        Parameters
        ----------
        item : str
        time : int
            Time index to be read
        sigma :int
            Sigma level index to be read
        Returns
        -------
        HoloViews.TriMesh
        '''

        # Reading pd.DataFrame of item using get.val() method 
        if len(self.variables[item].shape) == 1:   ### 1-D item (e.g.: depth) 
            df_item = self.get_val(item)
        elif len(self.variables[item].shape) == 2: ### 2-D item (e.g.: zeta)
            df_item = self.get_val(item, time=time)
        elif len(self.variables[item].shape) == 3: ### 3-D item (e.g.: salainity)
            df_item = self.get_val(item, time=time, sigma=sigma)
        else:
            print('ERROR: The shape of item is incorrect in self.__plot_2dh')
            sys.exit()
        return hv.TriMesh((self.tris, hv.Points(df_item)))

    def dmap(self, item, x_range=None, y_range=None, z_range=None, timestamp_location=None):
        '''
        Interactive plotting using DynamicMap
        Internal functions need to be deifned in order to be an argument of 
            functools.partial().
        Tips: Prepare an internal function,
            and set time and sigma at the end of the arguments in partial.
        内部関数を用意すること，partialの引数の最後をtimeとsigmaにするのがポイント
        
        Parameters
        ----------
        self : FVCOM2D instance
        item : str
        x_range : tuple
        y_range : tuple, optional
        z_range : tuple, optional
        timestamp_location : tuple, optional
        '''

        def item3d_plot(item, timestamp_location, sigma, time):
            '''
            Internal function for plotting 3D item using functools.partial().
            partialの引数とするため，内部関数とする.

            Parameters
            ----------
            item : str
            timestamp_location : tuple
            sigma : int
                sigma level
            time : int
                output timestep of time series data

            Returns
            -------
            hv.TriMesh * self.plot_timestamp
            '''

            item = self.get_val(item, time=time, sigma=sigma)
            timestamp_txt = self.plot_timestamp(time, timestamp_location)
            return hv.TriMesh((self.tris, hv.Points(item))) * timestamp_txt

        def item2d_plot(item, timestamp_location, time):
            '''
            Internal function for plotting 2D item (e.g. zeta) using functools.partial().
            
            Parameters
            ----------
            item : str
            timestamp_location : tuple
            time : int
            
            Returns
            -------
            hv.TriMesh * self.plot_timestamp
            '''

            item = self.get_val(item, time=time)
            timestamp_txt = self.plot_timestamp(time, timestamp_location)
            return hv.TriMesh((self.tris, hv.Points(item))) * timestamp_txt        
        
        if len(self.variables[item].shape) == 1:   ### 1-D item (ex: depth) 
            print(f"ERROR: 1-D item of {item} is not supported in FVCOM2D.dmap")
            sys.exit()
        elif len(self.variables[item].shape) == 2: ### 2-D item (ex: zeta)
            p = partial(item2d_plot, item, timestamp_location)
            dmap = hv.DynamicMap(p, kdims=['time']).redim.values(time=range(len(self.time)))
        elif len(self.variables[item].shape) == 3: ### 3-D item (ex: salainity)
            p = partial(item3d_plot, item, timestamp_location)
            dmap = hv.DynamicMap(p, kdims=['sigma', 'time']).redim.values( \
                sigma=range(len(self.siglay))).redim.values(time=range(len(self.time)))
        else:
            print('ERROR: The shape of the item is incorrect in FVCOM2D.dmap')
            sys.exit()

        if x_range is None:
            x_range = (self.x.min(), self.x.max())
        if y_range is None:
            y_range = (self.y.min(), self.y.max())
        if z_range is None:
            return rasterize(dmap, x_range=x_range, y_range=y_range)
        else:
            return rasterize(dmap.redim.range(**{item : z_range}), \
                             x_range=x_range, y_range=y_range)

    def splot_2dh(self, item, time=0, sigma=0, x_range=None, y_range=None,
                  z_range=None, timestamp_location=None, **kwargs):
        '''
        Static horizontal 2D plot (one sigma and time frame)
        静的な平面2次元プロット．InteractiveにはDynamicMapを用いたdmap()を使用する
        '''
        if x_range is None:
            x_range = (self.x.min(), self.x.max())
        if y_range is None:
            y_range = (self.y.min(), self.y.max())

        p = self.__plot_2dh(item=item, time=time, sigma=sigma)
        if z_range is None or z_range == False:
            p = rasterize(p, x_range=x_range, y_range=y_range)
        else:
            p = rasterize(p.redim.range(**{item : z_range}), x_range=x_range, y_range=y_range)

        timestamp_txt = self.plot_timestamp(time, timestamp_location)
        return p * timestamp_txt

    def plot_point(self, coords, color='red', marker='+', size=10, **kwargs):
        p = hv.Points(coords, **kwargs)
        return p.opts(color=color, marker=marker, size=size)
        
    def plot_timestamp(self, time=0, timestamp_location=None, **kwargs):
        '''
        timestamp_location: tuple (x, y)
        Currently setting the location using graphic space unsupported in Holoviews
        スクリーン座標系で位置が設定できるとよいが，現在はHoloviewsの仕様で不可
        timestamp_location=Noneのときは空文字を出力することで汎用化した
        '''
        if timestamp_location is None:
            return hv.Text(0, 0, "", **kwargs)
        else:
            timestamp = self.timestamp[time]
            return hv.Text(timestamp_location[0], timestamp_location[1], timestamp, **kwargs)

    def make_frame(self, item, title=None, time=None, sigma=None,
                   plot_mesh=True, plot_coastline=True,
                   x_range=None, y_range=None, z_range=None,
                   timestamp_location=None):
        '''
        Make a frame at one time step [and one sigma layer (only for 3D)]
        1つのframe出力．アニメーション作成に有効
        '''
        ptri = self.splot_2dh(item=item,time=time, sigma=sigma, colorbar_title=title,
                              x_range=x_range, y_range=y_range,
                              z_range=z_range, timestamp_location=timestamp_location)
        pmesh = self.plot_mesh(x_range=x_range, y_range=y_range)
        pcoastline = self.plot_coastline(x_range=x_range, y_range=y_range)
        #timestamp_txt = self.plot_timestamp(time, timestamp_location)

        if plot_mesh and plot_coastline:
            return ptri * pmesh * pcoastline
        elif plot_mesh:
            return ptri * pmesh
        elif plot_coastline:
            return ptri * pcoastline
        else:
            return ptri

class Fvcom2D_trans(Fvcom2D):
    '''
    Postprocessing for FVCOM 4 with converting coordinates
    Requires function proj_trans()
    (x, y)から(lon, lat)への変換を想定しているが，epsg0とepsg1と座標名を与えることで任意の変換が可能
    '''
    def __init__(self, col_x='Longitude', col_y='Latitude', epsg0="2451", epsg1="4612", \
                 ncfile_path='./output/tokyo_0001.nc', obcfile_path='./inputfile/tokyo_obc.dat'):
        '''
        m_to_km and offset must be False in super().__init__().
        '''
        super().__init__( ncfile_path, obcfile_path, m_to_km=False, offset=False)
        self.verts = proj_trans(df=self.verts, col_x=col_x, col_y=col_y, epsg0=epsg0, epsg1=epsg1)
        points = gv.operation.project_points(gv.Points(self.verts, vdims=['z']))
        self.mesh = du.mesh(self.verts, self.tris)
        self.trimesh = hv.TriMesh((self.tris, points))
        self.verts = points

In [ ]:
ncfile = 'tst_0001.nc'
obcfile = 'tst_obc.dat'

In [ ]:
fvcom = Fvcom2D(ncfile_path=ncfile, obcfile_path=obcfile, m_to_km=True, offset=False)

In [ ]:
#fvcom.FVCOM.variables['zeta'][0]
#fvcom.variables['zeta'][0]
#len(fvcom.variables['zeta'].shape)
fvcom.variables['zeta'][0].min()

In [ ]:
#fvcom.variables['zeta'][1].max()
np.where(fvcom.variables['zeta'][24] > 10)

# Static plotting by spacifying time (and sigma)
- **注意** `z`が一様分布のときは`z_range()`を指定しないと描画されない．
- **Caution:** No plotting when `z_range=None` and `z` is uniform. 
- `timestamp_location=` tuple `(x, y)` or `None`
- `toolbar_location=` `None` or `'top'` or `'east'` or `'west'`
- **Caution:** Plotting coastline may take considerable time.

In [ ]:
# Select item to be plotted.
item='salinity'; title = "Salinity (psu)"; z_range=(0,35)
#item='zeta'; title = "Surface elevation (m)"; z_range=(0,1)

# Set time and sigma level
time=10
sigma=0

# Set x and y ranges. = None for auto setting (you may change after plotting).
x_range=None
y_range=None

# Set timestamp_location. = None to deactivate
timestamp_location=(270, 148)

# Create color map.
smap = fvcom.splot_2dh(item=item, time=time, sigma=sigma,
                       x_range=x_range, y_range=y_range, z_range=z_range,
                       timestamp_location=timestamp_location)

# Create mesh.
mesh = fvcom.plot_mesh(line_color='red', line_width=0.1)

# Create coastline.
coastline = fvcom.plot_coastline(color='red', line_width=1)

# Overlay as you like
p = smap * mesh * coastline

# Apply customization options.
# Customize colorbar.
colorbar_opts=dict(title=title, title_text_font_size="16pt", 
                   title_text_baseline = "bottom",
                   title_standoff =10, title_text_font_style = "normal",
                   major_label_text_font_size = "14pt", major_label_text_align="left",
                   label_standoff=4, width=15)
# Customize Image plot.
#p_opts = opts.Image(tools=['hover'], cmap='viridis', colorbar_position='right',
#                    width=600, height=250, colorbar=True, colorbar_opts=colorbar_opts)
p_opts = opts.Image(tools=['hover'], cmap='viridis', colorbar_position='right',
                    frame_width=500, aspect='equal', colorbar=True, colorbar_opts=colorbar_opts)
p.redim.label(x='X (km)', y='Y (km)').opts(p_opts)

# You may further customize by appending .redim() and .opts() as follows:
#p.redim.label(x='X (km)', y='Y (km)').opts(p_opts).redim(
#    x=hv.Dimension('x', range=(260,280))).opts(xticks=10)

# Plot marks and mesh
- See [Points](https://holoviews.org/reference/elements/matplotlib/Points.html) for marker plots.

In [ ]:
# Create points at specified nodes
node1=10; node2=13
coords1 = [[fvcom.variables['x'][node1].item()/1000, fvcom.variables['y'][node1].item()/1000]]
coords2 = [[fvcom.variables['x'][node2].item()/1000, fvcom.variables['y'][node2].item()/1000]]

In [ ]:
opts.defaults(opts.Points(tools=['hover']))
#mesh = fvcom.plot_mesh(width=600, height=250, line_color='red', line_width=0.1)
mesh = fvcom.plot_mesh(line_color='red', line_width=0.1)
points1 = fvcom.plot_point(coords1, color='blue', marker='o', size=5)
points2 = fvcom.plot_point(coords2, color='green')
p = mesh * points1 * points2
p_opts = opts.Points(frame_width=500, aspect='equal', xlim=(255, 290))
p.opts(p_opts)

## Plot coastlines

In [ ]:
mesh * fvcom.plot_coastline() * points1 * points2

# Export to html
- Interactive plotting of the html on a browser when `toolbar='right'`. Try zoom and reload on the rowser; then export to PNG, which is high quality graphic file.
- Static plotting of the html on a browser when `toolbar=None`. This html may be further used to converting to PNG using pyppeteer, which may be useful for creating animation.
- Holoviews function of exporting to PNG did not work well.
- See [Working with Plot and Renderers](https://holoviews.org/user_guide/Plots_and_Renderers.html).

In [ ]:
# Set PNG output file path
dirpath = "./png/"
core = "tri_sal_"
# toolbar='right' for interactive view in a browser
#     Possible to zoom in on the browser and output png in it.
# toolbar=None to deactivate this interactivity. Useful for PDF export using pyppeteer.
toolbar=None # None, 'right'

# Select item to be plotted.
item='salinity'; title = "Salinity (psu)"; z_range=(0,35)
#item='zeta'; title = "Surface elevation (m)"; z_range=(0,1)

# Set time and sigma level
#time=10
sigma=0

# Set x and y ranges. = None for auto setting (you may change after plotting).
x_range=(252, 290) # None, (min, max)
y_range=(145, 165) # None, (min, max)

# Set timestamp_location. = None to deactivate
timestamp_location=(270, 148)

if not os.path.isdir(dirpath):
    os.makedirs(dirpath)

colorbar_opts=dict(title=title, title_text_font_size="16pt", 
                   title_text_baseline = "bottom",
                   title_standoff =10, title_text_font_style = "normal",
                   major_label_text_font_size = "14pt", major_label_text_align="left",
                   label_standoff=4, width=15)
p_opts = opts.Image(cmap='viridis', colorbar_position='right',
                    frame_width=500, aspect='equal', colorbar=True, colorbar_opts=colorbar_opts,
                    toolbar=toolbar)
                    #hooks=[fixBottomMargin])

for time in range(10):
    outputfile = f"{dirpath}{core}{time:03}"

    # Create color map.
    smap = fvcom.splot_2dh(item=item, time=time, sigma=sigma,
                           x_range=x_range, y_range=y_range, z_range=z_range,
                           timestamp_location=timestamp_location)
    smap=smap.redim.label(x='X (km)', y='Y (km)').opts(p_opts)

    # Create mesh.
    mesh = fvcom.plot_mesh(line_color='red', line_width=0.1)

    # Create coastline.
    coastline = fvcom.plot_coastline(color='red', line_width=1)

    # Overlay as you like
    p = smap * mesh * coastline
    hv.save(p, outputfile, fmt='html', backend='bokeh')
    # Direct exporting to PNG is possbile but resolution cannot be controlled.
    # You may increase the frame_width; then you also need to increase
    #     the size of font. You can avoid such inconvenience by exporting to HTML.
    # hv.save(p, outputfile, fmt='png', backend='bokeh')

## Load and plot HTML

In [ ]:
from IPython.display import HTML
from PIL import Image
file = display(HTML(filename = "/home/teem/Github/pyfvcom/examples/png/tri_sal_001.html"))

# Loading html and exporting to PNG using [pyppeteer](https://github.com/pyppeteer/pyppeteer)
## Example of handling web page

In [ ]:
import asyncio
import nest_asyncio
from pyppeteer import launch
from pyppeteer.browser import Browser
from pyppeteer.page import Page
# To avoid an error, the following should be invoked.
nest_asyncio.apply()

async def main():

    browser = await launch(headless=True)
    page = await browser.newPage()

    await page.goto('https://tool-taro.com/wget/')

    # URL入力
    await page.type('div.box_whois_l.box_wget03 > input[type="text"]',
                    'https://www.yahoo.co.jp/')

    # 取得ボタンクリック
    await page.click('div.box_whois_r > div > input')

    # 3秒待機
    await page.waitFor(3000)

    # タイトル
    element = await page.querySelector('#new > div.box_wget01 > textarea')
    title = await page.evaluate('(element) => element.value', element)

    # 本文
    element = await page.querySelector('#new > div.box_wget02 > textarea')
    description = await page.evaluate('(element) => element.value', element)

    print(title)
    print(description)

    # 画面キャプション
    await page.screenshot({'path': 'example.png'})
    await browser.close()

asyncio.get_event_loop().run_until_complete(main())

## Exporting from HTML to PNG using [pyppeteer](https://github.com/pyppeteer/pyppeteer)
Specify the original image `width` and `height`; then specify `clip` area to adjust the boundingbox in a trial and error manner. To inrease the resolution, specify `deviceScaleFactor`. 
This often results in a error of `connection unexpectedly closed`; thus this work should be done in bash shell (though still the problem occurs).

In [ ]:
async def main(html_url, png_path=None, pdf_path=None, width=1920, height=1080, 
                   deviceScaleFactor=5, waitFor=3000, clip=None):
    '''
    Loading HTML (webpage) and exporting to PNG with specifying DPR using pyppetter
    Tested on Ubuntu 20.04 LTS on WSL2
    https://pyppeteer.github.io/pyppeteer/reference.html#browser-class
    https://miyakogi.github.io/pyppeteer/reference.html#page-class
    https://github.com/erdewit/nest_asyncio
    
    Parameters
    ----------
    html_url : str
        e.g.: "file:///home/teem/Github/pyfvcom/examples/png/example.html"
    out_path : str
        Output PNG path
    width : int, optional
        Viewport width or original image width (adjust in a trial and error manner)
    height : int, optional
        Viewport height or original image height (adjust in a trial and error manner)
    deviceScaleFactor : int, optional
        DPR (Device Pixel Ratio) = DPI/72. E.g., 5 corresponding to 360 DPI
    waitFor : int, optional
        Wait for function, timeout, or element which matches on page (milliseconds)
    clip : dict, optional
        Clipping area of the page
        {"x"(int): x-coordinate of top-left corner of clipping area, "y"(int): y-coordinate, 
         "width"(int): width of clipping area, "height"(int): height}
    '''

    browser = await launch(headless=True)
    page = await browser.newPage()

    await page.goto(html_url)
    # 'deviceScaleFactor' = DPR (Device Pixel Ratio) = DPI/72
    # 'hasTouch': True => Activates reloading browser page so that scaling (zooming) with
    #                     increasing resolution being implemented.
    await page.setViewport({'width': width, 'height': height,
                            'deviceScaleFactor': deviceScaleFactor,
                            'hasTouch': True})
    await page.waitFor(3000)
    #if png_path is not None:
    # await page.screenshot({'path': out_path, 'scale':1})
    await page.screenshot(path=png_path, scale=1, clip=clip)
    #if pdf_path is not None:
    #    await page.pdf(path=pdf_path, scale=1, width=width, height=height)
    await browser.close()

html_url = "file:///home/teem/Github/pyfvcom/examples/png/tri_sal_000.html"
png_path  = "./png/tri_sal_000.png"
# PDF may be unstable (often becoming busy).
pdf_path  = None # "./png/tri_sal_001.pdf"

# Adjust width and height in a trial and error manner
width=700; height=340
clip = None # {"x": 20, "y": 10, "width": 600, "height": 250}
asyncio.get_event_loop().run_until_complete(main(html_url, png_path, pdf_path,
    width=width, height=height, clip=clip))

In [ ]:
# asyncio does not work well in for loop.
# Set PNG output file path
'''
dirpath = "./png/"
core = "tri_sal_"
# Adjust width and height in a trial and error manner
width=700; height=340
for time in range(1):
    html_url = f"file:///home/teem/Github/pyfvcom/examples/png/{core}{time:03}.html"
    png_path = f"{dirpath}{core}{time:03}.png"
    print(f"png_path={png_path}")
    print(f"html_url={html_url}")
    #loop = asyncio.get_event_loop()
    #loop.run_until_complete(main(html_url, png_path,
    #    width=width, height=height, clip=None))
    #loop.close()
    asyncio.get_event_loop().run_until_complete(main(html_url, png_path,
        width=width, height=height, clip=None))
'''

# Make APNG animation
- `apngasm64.exe output.png frame*.png 1 4`

In [ ]:
output_png="animation.png"
cmd = "apngasm64.exe" + " " + dirpath + output_png + " " + dirpath + core + "*.png" + " " + "1 1"
print(cmd)
subprocess.call(cmd, shell=True)
web="file:///home/teem/Github/pyfvcom/examples/png/" + output_png
print(web)
# Copy web in the browser, and an animation can be seen.

# Intaractive Plotting using DynamicMap
- An instance method of `self.dmap()` creates a dynamic map where `item` should be give.
- `x_range`, `y_range` and `z_range` are optional tuples. If not given, the ranges are automatically set as their full ranges.
- For annotating graph, edit the `%%opts` and `Mod()`.
- **Caution:** Cell magic of `%%opts` should be placed at the top in the cell.
- **Caution:** `%%output holomap='scrubber'` should be disabled in case of kdim containing `sigma`.
- `Mod()` is a method in [`HoloExt()`](https://holoext.readthedocs.io/en/latest/index.html). HoloExt source: c:\Users\jsasa\Miniconda3\Lib\site-packages\holoext\bokeh.py
- To overlay mesh, append ` * fvcom.plot_mesh(color='blue', line_width=0.1, x_range=x_range, y_range=y_range)`
- To overlay coastline, append ` * fvcom.plot_coastline(color='red', line_width=0.5, x_range=x_range, y_range=y_range)`
- **Caution:** Plotting coastline takes considerable time.

## Interactive plotting 2D item without sigma level

In [ ]:
# Select item to be plotted.
item='zeta'; title = "Surface elevation (m)"; z_range=(0,1)
# Set x, y, and z ranges. = None for auto setting (you may change after plotting).
x_range=None #(250,295)
y_range=None #(145,165)

# Set timestamp_location. = None to deactivate
timestamp_location=(270, 148)

# Create color map.
dmap_ds=fvcom.dmap(item=item, x_range=x_range, y_range=y_range, z_range=z_range, \
                   timestamp_location=timestamp_location)
# Create mesh.
mesh = fvcom.plot_mesh(line_color='red', line_width=0.1)

# Create coastline.
coastline = fvcom.plot_coastline(color='red', line_width=1)

# Overlay as you like
p = dmap_ds * mesh * coastline

# Apply customization options.
# Customize colorbar.
colorbar_opts=dict(title=title, title_text_font_size="16pt",
                   title_text_baseline = "bottom",
                   title_standoff =10, title_text_font_style = "normal",
                   major_label_text_font_size = "14pt", major_label_text_align="left",
                   label_standoff=4, width=15)
# Customize Image plot.
p_opts = opts.Image(tools=['hover'], cmap='viridis', colorbar_position='right',
                    frame_width=400, aspect='equal', colorbar=True, colorbar_opts=colorbar_opts)
p.redim.label(x='X (km)', y='Y (km)').opts(p_opts)

# You may further customize by appending .redim() and .opts() as follows:
#p.redim.label(x='X (km)', y='Y (km)').opts(p_opts).redim(
#     x=hv.Dimension('x', range=(260,280))).opts(xticks=10)

# You may save


## Interactive plotting 3D item with sigma level

In [ ]:
# Select item to be plotted.
item='salinity'; title = "Salinity (psu)"; z_range=(0,35)

# Set x, y, and z ranges. = None for auto setting (you may change after plotting).
x_range=None
y_range=None

# Set timestamp_location. = None to deactivate
timestamp_location=(270, 148)

# Create color map.
dmap_ds=fvcom.dmap(item=item, x_range=x_range, y_range=y_range, z_range=z_range, \
                   timestamp_location=timestamp_location)
# Create mesh.
mesh = fvcom.plot_mesh(line_color='red', line_width=0.1)

# Create coastline.
coastline = fvcom.plot_coastline(color='red', line_width=1)

# Overlay as you like
p = dmap_ds * mesh * coastline

# Apply customization options.
# Customize colorbar.
colorbar_opts=dict(title=title, title_text_font_size="16pt", 
                   title_text_baseline = "bottom",
                   title_standoff =10, title_text_font_style = "normal",
                   major_label_text_font_size = "14pt", major_label_text_align="left",
                   label_standoff=4, width=15)
# Customize Image plot.
p_opts = opts.Image(tools=['hover'], cmap='viridis', colorbar_position='right',
                    frame_width=400, aspect='equal', colorbar=True, colorbar_opts=colorbar_opts)
p.redim.label(x='X (km)', y='Y (km)').opts(p_opts)

# You may further customize by appending .redim() and .opts() as follows:
#p.redim.label(x='X (km)', y='Y (km)').opts(p_opts).redim(
#     x=hv.Dimension('x', range=(260,280))).opts(xticks=10)

# Enforcing plot customization using bokeh's primitive function
- It is possible to enforce plot customization by modifying plot objects directly using bokeh's primitive functions. See [**Customizing Plots**](https://holoviews.org/user_guide/Customizing_Plots.html).
- See available options of HoloViews by `hv.help(hv.Image)`
- Further customization is possible using bokeh's primitive functions by `hooks=[hook]`:

```Python
def hook(plot, element):
    '''
    Enforce plot customization by modifying plot object directly using bokeh.
    Edit as you like.
    
    Parameters
    ----------
    plot : HoloViews plot object
    element : Element being rendered
    '''
   
    plot.state.axis.axis_line_width = 1
    plot.state.axis.axis_label_text_font_style = 'normal'
```

In [ ]:
def hook(plot, element):
    '''
    Enforce plot customization by modifying plot object directly using bokeh.
    Edit as you like.
    
    Parameters
    ----------
    plot : HoloViews plot object
    element : Element being rendered
    '''

    #plot.state.outline_line_color = 'black'
    #plot.state.outline_line_width = 2
    plot.state.axis.axis_line_width = 1.5
    plot.state.axis.axis_label_text_font_style = 'normal'
    ### plot.state.x_range = models.Range1d(0,22) # Do not apply: Dynamic resolution change disabled.
    ### plot.state.y_range = models.Range1d(0,33)

In [ ]:
# Customize Image plot using hooks.
# See the axes label's font style becomeing normal and axes lines thicker .
p_opts = opts.Image(tools=['hover'], cmap='viridis', colorbar_position='right',
                    frame_width=400, aspect='equal', colorbar=True, colorbar_opts=colorbar_opts,
                    hooks=[hook])

p.opts(p_opts).redim.label(x='X (km)', y='Y (km)')